# 네이버 영화리뷰 감성 분석

### 데이터 로드

In [7]:
# 데이터 다운로드
from tensorflow.keras.utils import get_file

review_train_path = get_file("ratings_train.txt", "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt")
review_test_path = get_file("ratings_test.txt", "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt")

In [8]:
# 데이터프레임 생성
import pandas as pd 

review_train_df = pd.read_csv(review_train_path, sep="\t")
review_test_df = pd.read_csv(review_test_path, sep="\t")

display(review_train_df)
display(review_test_df)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


### 데이터 전처리

In [ ]:
# 결측치 제거
review_train_df.isna().sum()

review_train_df = review_train_df.dropna(how='any')
review_test_df = review_test_df.dropna(how='any')

In [50]:
review_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [74]:
import re
from konlpy.tag import Okt
from nltk.stem import PorterStemmer
from tqdm import tqdm

def preprocess_review(text):
    okt = Okt()
    stemmer = PorterStemmer()
    stop_words = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

    preprocessed_sentences = []
    vocab = {}

    text = re.sub('[^ㄱ-ㅎ가-핳a-zA-Z0-9 ]', '', sentence)
    tokens = okt.morphs(text, stem=True)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    tokens = [token for token in stemmed_tokens if token not in stop_words]

    # vocab 생성
    for token in tokens:
        if token not in vocab:
            vocab[token] = 1
        else:
            vocab[token] += 1

    preprocessed_sentences.append(tokens)

    return preprocessed_sentences, vocab

In [77]:
preprocessed_train, train_vocab = preprocess_review(review_train_df['document'])
preprocessed_test, test_vocab = preprocess_review(review_test_df['document'])


In [67]:
# 역순 정렬
vocab_sorted = sorted(vocab.items(), key=lambda item: item[1], reverse=True)

# 인덱스 단어사전 생성
idx_to_word = {i+1: word for i, (word, cnt) in enumerate(vocab_sorted)}

# OOV 처리
word_to_idx['OOV'] = len(word_to_idx) + 1 


In [78]:
# 수열처리(= 정수 인코딩)
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=15, oov_token='<OOV>')
tokenizer.fit_on_texts(preprocessed_train)
train_sequences = tokenizer.texts_to_sequences(preprocessed_train)
test_sequences = tokenizer.texts_to_sequences(preprocessed_test)

In [80]:
# 패딩
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_padded = pad_sequences(train_sequences, padding='post', maxlen=50, truncating='post')
test_padded = pad_sequences(test_sequences, padding='post', maxlen=50, truncating='post')

### 모델 정의 및 생성

In [ ]:
from tensorflow.keras.datasets import imdb
import torch
import torch.nn.functional as F

VOCAB_SIZE = 300    # 사용할 단어 수
SEQ_LEN = 150       # 시퀀스 하나의 최대 길이

(train_input, train_target), (test_input, test_target) = imdb.load_data(num_words=VOCAB_SIZE)

# Torch Tensor 변환
train_input = [torch.tensor(seq, dtype=torch.long) for seq in train_input][:10000]
test_input = [torch.tensor(seq, dtype=torch.long) for seq in test_input][:5000]

train_target = torch.tensor(train_target, dtype=torch.long)[:10000]
test_target = torch.tensor(test_target, dtype=torch.long)[:5000]

# 패딩 처리
def pad_sequences(sequences, maxlen, padding_value=0):
    padded_sequences = [F.pad(seq[:maxlen], (0, max(0, maxlen-len(seq))), value=padding_value) for seq in sequences]
    return torch.stack(padded_sequences)

NameError: name 'vocab_size' is not defined

### 모델 학습

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

### 모델 추론

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')